### 先處理NER在進行關係分析。

In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


# 定義 CoT NER Prompt
ner_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
    這是一個命名實體識別（NER）任務，你需要將文本中的實體分類為以下類別：
    ### **命名實體分類方式：**  
    - 人物（Person）：包括個人姓名、稱號、官職、別名等。
    - 時間（Date/Time）：歷史年代、具體年份、月份、日期、時刻等。  
    - 組織（Organization）：政府機構、學術機構、軍事組織、社會團體、企業等。
    - 事件（Event）：戰爭、革命、條約、政策變遷、災難、運動等。
    - 專有名詞（Proper Noun）：包含特定歷史文件、法律條文、計畫名稱等。
    - 數量（Quantity）：具體數字、統計數據、人口數、傷亡數等。
    - 貨幣（Money）：歷史貨幣單位及金額。
    - 比例（Percentage/Ratio）：百分比、比率、分數等。
     
    直接輸出 **JSON **格式，每個句子或事件應該是陣列中的一個獨立 JSON 物件。請勿包含其他文字、步驟或 Markdown 格式標記（例如 ```json）。  
    
    ### **文本輸入範例：愛因斯坦是20世紀最重要的科學家之一，其創立了現代物理學的兩大支柱的相對論及量子力學，有「現代物理學之父」之譽。1933年10月回到美國後，愛因斯坦成為普林斯頓高等研究院的常駐教授。** 
        
    ### **以下是輸出格式範例：**  
    {{
        "人物": ["愛因斯坦"],
        "時間": ["20世紀"],
        "地點": [],
        "組織": [],
        "專有名詞": ["相對論","量子力學","現代物理學之父"]
        "數量": [],
        "貨幣": [],
        "比例": [],
        "事件": ["創立相對論與量子力學"]
    }},
    {{
        以此類推...
    }}
    ### **範例到此結束。**
    
    將輸入每一段輸入的文本，提取成命名實體：
    {text}
    """
)



In [2]:
# 定義 CoT 關係識別 Prompt
relation_prompt = PromptTemplate(
    input_variables=["entities"],
    template="""
    請分析以下實體，推理它們之間的關係。
    請使用 **逐步推理（Chain-of-Thought, CoT）** 的方式：
    1. 觀察每個實體類別（人物、組織、事件等），並思考它們可能的關聯。
    2. 嘗試基於以下wikidata常見的屬性(Property)推理可能的關係：
        人物關係（Person） 相關屬性：
        - P31 (instance of)：實體所屬的類別，例如：「孫中山」- P31 - 「人類」
        - P27 (country of citizenship)：國籍，例如：「孫中山」- P27 - 「中華民國」
        - P106 (occupation)：職業，例如：「孫中山」- P106 - 「政治家」
        - P39 (position held)：職位，例如：「孫中山」- P39 - 「中華民國臨時大總統」
        - P50 (author)：作者，例如：「愛因斯坦」- P50 - 「相對論」    
        - P112 (founded by)：創辦者，創辦機構的人物或組織。
        組織(Organization)關係相關屬性： 
        - P199 (organizational divisions)：業務部門，本組織的組織部門（不是獨立的法人實體）。
        - P241 (military branch)：軍種或部隊，該軍事單位、獎項、辦公室或個人所屬的部門，例如皇家海軍。
        - P27 (country of citizenship)：國籍，承認主體是其公民的客體國家。
        - P31 (instance of)：隸屬於，此主體是其中特定示例及成員的類別；與上級分類（P279）不同；例如：富士山隸屬於山，火山上級分類是山（且隸屬於火山地貌）"
        - P355 (has subsidiary)：子組織，一家公司或組織的下屬機構部門；通常是完全擁有的單立集團。與「業務部門」（P199）相對應。母組織（P749）的逆屬性。
        - P361 (part of)所屬實體，將主體視為其中一部分的客體（如果該主體已經是客體A的一部分，而客體A又是客體B的一部分，則請只將主體設為客體A的一部分），是「可分為」（P527）的逆屬性（另見「擁有此類別的一部分」（P2670））
        - P366 (has use)：用途，該項目的主要用途（含當前及過去）。
        - P749 (parent organization)：母組織，一個組織的母方組織，是子組織（P355）的逆屬性。
        - P1448 (official name)：官方名稱，以官方語言表示的主題的官方名稱。
        - P92 (main regulatory text)：主要規管文獻，當主體受管制時，用以設立主要管理規則的文本。
        - P4428 (implementation of)	：主實作，標準、程式、規範或程式語言的實作。
        數字概念（Numbber Date/Time）相關屬性：
        - P4876 number of records ：記錄數量，記錄的數量。
        - P2067	(mass)：重量，例如：「某艦艇」- P2067 - 「5000噸」 。
        - P38 (currency)：貨幣，項目使用的貨幣
        - P1107	(Percentage/Ratio)：比例，在整體中所占的份份額。
        - P1181	(numeric value)： 數字，某數字（例：1）或常數（圓周率）的數值。        
        時間概念（Date/Time）相關屬性： 
        - P577 (publication date)：出版日期，作品首次出版或發行的時間。
        - P580 (start time)：始於，某項開始存在的時間或聲明開始有效的時間。
        - P582 (end time)：終於，某項停止存在的時間或聲明不再有效的時間。
        - P585 (point in time)：時間點，某次事件發生、存在，或是某陳述正確的日期。     
        事件（Event）相關屬性：
        - P361 (part of)：事件的一部分，例如：「二戰」- P361 - 「太平洋戰爭」
        - P793 (significant event)：重要事件，例如：「二戰」- P793 - 「珍珠港事件」    
        - P1344	(participant in)：參與，某、某機構、某團體或某人參與的事情，P710或P1923的逆屬性。
        - P710	(participant)："參與者"，參與/參加活動的人，團體或組織（對象）（主題）。
        - P1891 (signatory)：簽署方，簽署了正式文件的人、國家或组組織。               
        地理(location/geography)概念相關屬性：
        - P361 (part of)：地點的一部分，例如：「基隆」- P361 - 「台灣」
        - P159 (headquarters location)：組織總部的所在地。	
        - P276 (location)客體、結構或活動的位置。
        - P740 (location of formation)：成立地點，組織成立的地點。    
        出版品(publication/book)相關屬性：
        - P1343 (described by source)：記載處，有記載此項的文獻等出處。
        - P144 (based on) 改編自，以主体項目為基礎的作品。
        - P1476 (title) 標題，作品的標題、作品（例如書、電影或網站）的名稱。
        - P50 (author) 作者，書面作品的主要創作者。
        
    3. 嘗試根據這些關係屬性，建立關係結構，**主體-關係-客體**。例如：
        - 如果一個人物與事件相關，他可能是 **發起** 或 **參與** 了該事件。
        - 如果一個組織與地點相關，這個組織可能 **位於** 該地點。
        - 如果一個人物與時間相關，這個人物可能 **出生於** 或 **死亡於** 這個時間。

    4. ***請確保輸出結果是標準的 JSON 格式，不包含任何其他文字。***

    格式範例如下:
    輸入實體:
    {{
        "人物": ["愛因斯坦"],
        "時間": ["20世紀"],
        "地點": [],
        "組織": [],
        "專有名詞": ["相對論","量子力學","現代物理學之父"]
        "數量": [],
        "貨幣": [],
        "比例": [],
        "事件": ["創立相對論與量子力學"]
    }},
    {{
        以此類推...
    }}    
    
    輸出關係結構:
    {{
         {{"事件": ["創立相對論及量子力學"]}},{{["主體": "愛因斯坦|人物", "關係": "時間|P585", "客體": "20世紀|時間"],["主體": "愛因斯坦|人物", "關係": "作者|P50", "客體": "相對論|專有名詞"],["主體": "愛因斯坦|人物", "關係": "作者|P50", "客體": "量子力學|專有名詞"]}}
        ,{{"事件": ["移居美國，任職於普林斯頓"]}},{{["主體": "愛因斯坦|人物", "關係": "時間|P585", "客體": "1933年10月|時間"],["主體": "愛因斯坦|人物", "關係": "任職|P108", "客體": "普林斯頓高等研究院|組織"]}}
        以類此類推...
        
    }}
    ### **範例到此結束。**
    
    將輸入實體清單，分析推理後建立成關係結構：
    {entities}
    """
)



In [3]:
import json
import re

# 執行 NER 任務
def extract_entities(text):
    raw_result = ner_chain.run(text)  
    #raw_result = ner_chain.invoke(text)  
    try:
        return json.loads(clean_json_output(raw_result))
    except json.JSONDecodeError:
        return {"error": "NER 解析錯誤", "raw_result": raw_result}

# 執行關係識別任務
def extract_relationships(entities):
    raw_result = relation_chain.run(json.dumps(entities, ensure_ascii=False))
    #raw_result = relation_chain.invoke(json.dumps(entities, ensure_ascii=False))
    try:
        return json.loads(clean_json_output(raw_result))
    except json.JSONDecodeError:
        return {"error": "關係識別解析錯誤", "raw_result": raw_result}

# 清理 JSON 輸出，移除 Markdown 標記
def clean_json_output(raw_output):
    # 移除可能的 Markdown JSON 標記
    cleaned = re.sub(r"```json\s*([\s\S]*?)\s*```", r"\1", raw_output).strip()
    return cleaned

# 安全地解析 JSON 字串
def safe_json_loads(data):
    if isinstance(data, str):
        try:
            return json.loads(data)  # 嘗試解析 JSON 字串
        except json.JSONDecodeError:
            print("錯誤：無法解析 JSON 字串")
            return {}  # 解析失敗時返回空字典
    return data  # 如果本來就是字典，則直接返回

# 將文本進行 NER 和關係識別
def process_text(text):
    # 先執行 NER
    entities = extract_entities(text)
    
    # 如果 NER 解析出錯，直接返回
    if "error" in entities:
        return entities
    
    # 再執行關係識別
    relationships = extract_relationships(entities)
    
    return {
        "entities": entities,
        "relationships": relationships
    }

In [30]:
import langchain
# 設定 debug 模式為 False
#langchain.debug = True
langchain.debug = False

print(langchain.__version__)

0.3.14


In [ ]:
# # 測試文本
# #text = "孫中山在1911年發起了辛亥革命，並創立了中國國民黨，該黨在南京建立政府。"
# text = "依據民國32年所發表的開羅宣言，日本應無條件投降並將台灣、澎湖群島歸還中國。"
# #result = process_text(text)
# print(json.dumps(process_text(text), indent=2, ensure_ascii=False))

In [4]:
# 設定基本的 LLM 模型

from langchain_openai import ChatOpenAI
import os

# 設置 LLM 模型
#API#1
#os.environ["GROQ_API_KEY"] = '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'
#API#2
#os.environ["GROQ_API_KEY"] = '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'
#API#3
#os.environ["GROQ_API_KEY"] = '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'
#API#4
#os.environ["GROQ_API_KEY"] = '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'
#API#5
os.environ["GROQ_API_KEY"] = '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'

llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=os.environ['GROQ_API_KEY'],
    model_name="llama-3.3-70b-versatile",
    #model_name="llama-3.2-3b-preview",        
    #model_name="llama-3.2-3b-preview",
    #model_name="mixtral-8x7b-32768",
    #model_name="llama-3.2-11b-vision-preview",
    #model_name="deepseek-r1-distill-qwen-32b",
    #model_name="deepseek-r1-distill-llama-70b",
    temperature=0.0,
  #  max_tokens=1000,
)

In [6]:
# # 測試文本
ner_chain = LLMChain(prompt=ner_prompt, llm=llm)
#test_text = "依據民國32年所發表的開羅宣言，日本應無條件投降並將台灣、澎湖群島歸還中國。因此，國民政府乃於民國33年5月在中央設計局之下設立台灣調查委員會，做為戰後接收台灣之準備。民國34年8月29日，國民政府特任陳儀為台灣省行政長官，負台灣接收及軍政全責；9月7日，政府又任命陳儀兼任台灣省警備總司令。"
#test_text = "光復之初，民眾表現出對國民政府熱烈的歡迎與支持。但由於二次大戰末期，臺灣屢遭美軍轟炸，公共設施遭受相當破壞，戰後生產原料取得不易，技術人才難求，短期內恢復戰前生產水準，自非易事。然而當時中央政府派遣來台灣的官員，集行政、司法、立法、軍事大權於一身的行政長官公署制，非但未能有效復原，臺政卻每況愈下，迭受抨擊，加上溝通不夠與觀念的差異，以及所採取的不良經濟、統治方式與風評不佳的官威軍政，種下釀成日後228事件的遠因。"
test_text = ["「霧社事件」發生在民國19年，由霧社地區德哥塔雅原住民賽德克的泰雅人揭竿起義，反抗日本帝國主義侵害行為之不幸事件，依據目睹該事件悲慘景象之巫金墩所述(詳下列檔案影像)，該事件係當時在霧社公校舉行例行運動大會時，由莫那魯道酋長率領部落勇士，全副武裝衝入運動場，殺死運動場上約一百六十餘個日本人，導致台灣總督府採取強力的軍事行動，使用國際所禁用的方式派飛機投擲化學毒氣，以慘無人道的手法殘害原住民。除事件領導人莫那魯道自縊外，該族族民亦幾遭滅族。",
    "巫金墩指出，霧社事件係起因於當時的日本完全忽視泰雅族的習慣，除屢次徵召原住民義務勞動苦役、欺壓、虐待外，日本警官亦凌辱部落婦女，加上「以蕃制蕃」措施，長期以來的種族衝突，讓原住民在無法忍受的怨恨累積下，點燃其反日情節，終致霧社事件的發生。",
    "當時的台灣總督府不但無法迅速處理因應，甚至採取不人道的手段鎮壓，因而遭到日本帝國議會強烈質疑，總督石塚英藏與總務長官人見次郎遭到撤換，嚴重影響軍國主義天皇思想的威信，總督府高壓式的「理蕃」政策也因此被迫做適度的調整修正；霧社事件當可說是日本統治台灣期間最後一次激烈的台灣反抗行動。",
    "電影是人類文化的新興載體，可直接記錄與再現人類生活歷史及自然界種種現象之原貌，透過電影手法將歷史檔案普及化的模式，正是導演籌拍史詩電影，將原住民抗日精神加以重現的另一種典範，令人期待之餘，我們已先從國家檔案一窺端倪囉。"
]
response = ner_chain.invoke({"text": test_text})
print(response)

{'text': '[\n    {\n        "人物": ["莫那魯道", "巫金墩", "石塚英藏", "人見次郎"],\n        "時間": ["民國19年"],\n        "組織": ["日本帝國", "台灣總督府"],\n        "專有名詞": ["霧社事件", "泰雅人", "賽德克", "霧社公校"],\n        "數量": ["一百六十餘個"],\n        "貨幣": [],\n        "比例": [],\n        "事件": ["霧社事件", "運動大會", "軍事行動"]\n    },\n    {\n        "人物": ["巫金墩"],\n        "時間": [],\n        "組織": ["日本帝國"],\n        "專有名詞": ["霧社事件", "泰雅族", "以蕃制蕃"],\n        "數量": [],\n        "貨幣": [],\n        "比例": [],\n        "事件": ["霧社事件", "種族衝突"]\n    },\n    {\n        "人物": ["石塚英藏", "人見次郎"],\n        "時間": [],\n        "組織": ["台灣總督府", "日本帝國議會"],\n        "專有名詞": ["霧社事件", "理蕃政策"],\n        "數量": [],\n        "貨幣": [],\n        "比例": [],\n        "事件": ["霧社事件", "撤換"]\n    },\n    {\n        "人物": [],\n        "時間": [],\n        "組織": [],\n        "專有名詞": ["電影", "史詩電影"],\n        "數量": [],\n        "貨幣": [],\n        "比例": [],\n        "事件": ["霧社事件"]\n    }\n]'}


In [ ]:
# # 建立 LLMChain
# ner_chain = LLMChain(prompt=ner_prompt, llm=llm)
# # 建立關係識別 LLMChain
# relation_chain = LLMChain(prompt=relation_prompt, llm=llm)

In [ ]:
import json
import os
import shutil


# 建立 LLMChain
ner_chain = LLMChain(prompt=ner_prompt, llm=llm)
# 建立關係識別 LLMChain
relation_chain = LLMChain(prompt=relation_prompt, llm=llm)



json_dir = "./docs/output/1_clean_json/llama/"
done_dir = "./docs/output/1_clean_json/llama/done/"
out_dir = "./docs/output/3_ner_re_p/llama_v7/"

# 確保 json_done 目錄存在
#os.makedirs(done_dir, exist_ok=True)
#os.makedirs(out_dir, exist_ok=True)


json_files = [f for f in os.listdir(json_dir) if f.endswith(".json")]

for filename in json_files:
    file_path = os.path.join(json_dir, filename)
    
    try:
        # 讀取 JSON 檔案
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        print(f"✅ 開始處理: {filename}")
        # 取得 articles 欄位並進行處理
        articles = data.get("articles", [])

        processed_articles = process_text(articles)
        
        # 將處理後的結果寫回 JSON
        output_data = {
            #輸出加入articles內容
            #"articles": articles,
            "result": processed_articles
        }
        
        out_path = os.path.join(out_dir, filename)
        
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(output_data, f, indent=2, ensure_ascii=False)
        
        print(f"✅ 已處理並存入: {out_path}")
        
        # 搬移已處理的 JSON 檔案到 json_done 目錄
        shutil.move(file_path, os.path.join(done_dir, filename))
        print(f"📂 已搬移 {filename} 到 {done_dir}")
        
    except json.JSONDecodeError:
        print(f"⚠️ JSON 解析錯誤: {filename}")
    except IOError as e:
        print(f"⚠️ 無法讀取檔案 {filename}，錯誤: {e}")
    print("-" * 50)
    
print("\n📂 全部 JSON 檔案處理完成！")



